### Code for performing Bayesian Inference
### The induction data of G102D-Y42M-I57N is used here for postsample inference as an example

In [2]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
from scipy.stats import halfnorm
from scipy.stats import sem
# from scipy.statistics import stdev
import os
import seaborn as sns
import sys
sns.set()

C_ATC = [0 ,10 ,20,40 ,60 ,80 ,100 ,150 ,250 ,500 ,750,1000]

In [4]:
def fold_change(lkns, el, gama,c, K=1):
    
    pref= 1/lkns -1
    numeratorfc = 1+np.exp(-el-gama)*((c/K)**2)
    denomenatorfc = 1+np.exp(-el)*((c/K)**2)
    
    FC = 1/(1+pref*numeratorfc/denomenatorfc)
    return FC

In [5]:
el_avg, el_std = 5.5,2.5
gama_avg, gama_std = 5, 2.5
sigma_std=0.05

In [9]:
FCs_read = []
with open('FCdata.txt','r') as g: fclines=g.readlines()[0].split('$')[1:-1]
for each in fclines:
    tempfc=[]
    for eac in each[2:-2].split('], ['):
        tpfc=[float(ea) for ea in eac.split(',')]
        tempfc.append(tpfc)
    FCs_read.append(tempfc)
print(len(FCs_read))

23


In [12]:
# experimental leakiness of each mutant and the corresponding sem
with open('leak_info.txt','r') as g:
    alllines=g.readlines()[0][2:-2]
    
temp_leakraw, Leakns_read =alllines.split('], ['), []
for each in temp_leakraw:
    tplkr, templ =each.split(','), []
    for eac in tplkr: templ.append(float(eac))
    Leakns_read.append(templ)
print(np.shape(Leakns_read))

(23, 2)


In [18]:
leakWT=Leakns_read[-1][0]

In [10]:
def transformto_12_4(allset,rep=4):
    resulth=[]
    for i in range(12):
        temph=[]
        for j in range(rep):
            if allset[j][i] == 0.0: pass
            else:
                temph.append(allset[j][i])
        resulth.append(temph)
    return resulth 

In [11]:
G102D_Y42M_I57N_data = transformto_12_4(FCs_read[13])
print(np.shape(G102D_Y42M_I57N_data))

(12, 4)


In [6]:
def posterior_prob(lk,elp,gamap,sigma,dataset):
    prob=1/1e200
    for i in range(1,len(dataset)):
        u = fold_change(lk,elp,gamap,C_ATC[i])
        for j in range(len(dataset[i])):
            p=np.exp(-(dataset[i][j]-u)**2/2/(sigma**2))/sigma
            prob*=p
    
    return prob

In [7]:
def generate_MC_para():
    eli=np.random.normal(el_avg,el_std)
    gamai=np.random.normal(gama_avg,gama_std)
    sigmai=halfnorm.rvs(loc=0,scale=sigma_std)
    return [eli,gamai,sigmai]

In [31]:
def MC_posterior_sampling(lkn, dataset, move, gamafix=False):
    infered_para=[]
    infered_paras_postprob=[]
    
    parai=generate_MC_para()
    while posterior_prob(lkn, parai[0],parai[1],parai[2],dataset)==0:
        parai=generate_MC_para()
        
    if gamafix: parai[1]=gamafix ####
        
    infered_para.append(parai)
    infered_paras_postprob.append(posterior_prob(lkn, parai[0],parai[1],parai[2],dataset))
    
    for i in range(move):
        if i%100000==0: print(i, flush=True)
        para_new = infered_para[-1].copy()
        
        # change el
        para_new[0] = np.random.normal(el_avg,el_std)
        probnew = posterior_prob(lkn, para_new[0],para_new[1],para_new[2],dataset)
        if infered_paras_postprob[-1]<=probnew:
            infered_para.append(para_new)
            infered_paras_postprob.append(probnew)
        else:
            rand=np.random.uniform(0,1)
            if rand<= probnew/infered_paras_postprob[-1]:
                infered_para.append(para_new)
                infered_paras_postprob.append(probnew)
            else:
                infered_para.append(infered_para[-1])
                infered_paras_postprob.append(infered_paras_postprob[-1])
                
        # change gama
        if not gamafix:
        
            para_new = infered_para[-1].copy()

            para_new[1] = np.random.normal(gama_avg,gama_std)
            probnew = posterior_prob(lkn, para_new[0],para_new[1],para_new[2],dataset)
            if infered_paras_postprob[-1]<=probnew:
                infered_para.append(para_new)
                infered_paras_postprob.append(probnew)
            else:
                rand=np.random.uniform(0,1)
                if rand<= probnew/infered_paras_postprob[-1]:
                    infered_para.append(para_new)
                    infered_paras_postprob.append(probnew)
                else:
                    infered_para.append(infered_para[-1])
                    infered_paras_postprob.append(infered_paras_postprob[-1])
                
        # change sigma
        para_new = infered_para[-1].copy()
        
        para_new[2] = halfnorm.rvs(loc=0,scale=sigma_std)
        probnew = posterior_prob(lkn, para_new[0],para_new[1],para_new[2],dataset)
        if infered_paras_postprob[-1]<=probnew:
            infered_para.append(para_new)
            infered_paras_postprob.append(probnew)
        else:
            rand=np.random.uniform(0,1)
            if rand<= probnew/infered_paras_postprob[-1]:
                infered_para.append(para_new)
                infered_paras_postprob.append(probnew)
            else:
                infered_para.append(infered_para[-1])
                infered_paras_postprob.append(infered_paras_postprob[-1])
                
    return [infered_para,infered_paras_postprob]

In [32]:
print(Leakns_read[13][0])

0.04230297709522358


In [33]:
mcstep=1000000
postsample_G102D_Y42M_I57N=[]
postsample_G102D_Y42M_I57N.append(MC_posterior_sampling(0.0423, G102D_Y42M_I57N_data,mcstep))

0
100000
200000
300000
400000
500000
600000
700000
800000
900000


In [34]:
print(np.shape(postsample_G102D_Y42M_I57N[0][0]))

(3000001, 3)


In [35]:
def get_infrd_para(postsample, step=3000):
    infrdpara=[]
    for each in postsample[0][0][::step]: infrdpara.append(each)
    
    return infrdpara[:]

In [36]:
def check_parainfer(lkinfo, paraminfered,percentile):
    perc=1-percentile
    
    ed= -np.log(1/lkinfo[0]-1) + np.log(1/leakWT-1)
    edl=-np.log(1/(lkinfo[0]-lkinfo[1])-1) + np.log(1/leakWT-1)
    edh=-np.log(1/(lkinfo[0]+lkinfo[1])-1) + np.log(1/leakWT-1)
        
    parainfered_el, parainfered_gama, parainfered_sigma = [],[],[]

    for i in range(len(paraminfered)):
        parainfered_el.append(paraminfered[i][0])
        parainfered_gama.append(paraminfered[i][1])
        parainfered_sigma.append(paraminfered[i][2])

    parainfered_el=np.sort(parainfered_el)
    parainfered_gama=np.sort(parainfered_gama)
    parainfered_sigma=np.sort(parainfered_sigma)
    
    median2 = [ [ed,edl,edh],
               [np.median(parainfered_el),parainfered_el[round(len(paraminfered)*perc/2)],parainfered_el[-round(len(paraminfered)*perc/2)]],
               [np.median(parainfered_gama),parainfered_gama[round(len(paraminfered)*perc/2)],parainfered_gama[-round(len(paraminfered)*perc/2)]],
             [np.median(parainfered_sigma),parainfered_sigma[round(len(paraminfered)*perc/2)],parainfered_sigma[-round(len(paraminfered)*perc/2)]]]
    return median2

In [37]:
check_parainfer(Leakns_read[13], get_infrd_para(postsample_G102D_Y42M_I57N), 0.95)

[[1.6272249724030479, 1.6145204832245068, 1.639783409862794],
 [8.20603435455774, 8.08428677247967, 8.324977880974918],
 [2.8274569134445464, 2.7707660398212393, 2.8823038653828084],
 [0.018115072632262776, 0.014642816422286884, 0.02309121197177842]]